In [51]:
#STREAMS2TABLE
#written by Zachary J.J. Roper

#A script for converting raw event codes and timestamps into a trial-item dataset

In [52]:
#This notebook requires the following modules
import os
import numpy
import csv

In [53]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [54]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [55]:
events.count('2756') #catch trials

5

In [56]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [57]:
#uberlist

In [58]:
'2810' in uberlist[74][2]

False

In [59]:
RT = [None]*len(pairedlist)
RTbeg = 0*len(pairedlist)
RTend = 0*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(i,trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [60]:
#RT

In [61]:
#The following provides indexed lists for each of the individual event codes of interest 

In [62]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [63]:
#float(times[74])-float(times[70])

In [64]:
#Saccade

In [65]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [66]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [67]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [68]:
#print(TimeTrials[0])

In [69]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [70]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0

In [71]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [72]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [73]:
#RT

In [74]:
checksum = sum(error)+sum(abort)+sum(catch)

In [75]:
checksum

867

In [76]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 0 0 0 0 0 0
2 1 0 0 0 0 0
3 1 0 0 0 0 0
4 0 1 0 1 0 217.98919999999998
5 0 1 0 1 0 181.9854000000014
6 0 1 0 1 0 173.9981000000007
7 1 0 0 0 0 0
8 1 0 0 0 0 0
9 0 0 0 1 0 195.95260000000053
10 0 0 0 1 0 210.0020000000004
11 0 1 0 1 0 193.9867000000013
12 0 1 0 1 0 181.98539999999775
13 1 0 0 0 0 0
14 0 1 0 1 0 197.91889999999694
15 0 0 0 1 0 163.96310000000085
16 0 1 0 1 0 187.96540000000095
17 0 1 0 1 0 216.02299999999377
18 0 1 0 1 0 191.97960000000603
19 1 0 0 0 0 0
20 1 0 0 0 0 0
21 1 0 0 0 0 0
22 1 0 0 0 0 0
23 1 0 0 0 0 0
24 1 0 0 0 0 0
25 1 0 0 0 0 0
26 1 0 0 0 0 0
27 0 1 0 1 0 164.04490000000078
28 0 1 0 1 0 191.97939999999653
29 0 1 0 1 0 177.9710999999952
30 1 0 0 0 0 0
31 0 1 0 1 0 119.9716999999946
32 1 0 0 0 0 0
33 1 0 0 0 0 0
34 1 0 0 0 0 0
35 1 0 0 0 0 0
36 1 0 0 0 0 0
37 1 0 0 0 0 0
38 0 1 0 1 0 60.00639999999839
39 1 0 0 0 0 0
40 1 0 0 0 0 0
41 1 0 0 0 0 0
42 0 0 0 1 0 156.0165999999881
43 1 0 0 0 0 0
44 0 1 0 1 0 205.98769999999786
45 1 0 0 0 0 0
46 0 1 0 1 0 181.98

669 1 0 0 0 0 0
670 1 0 0 0 0 0
671 1 0 0 0 0 0
672 1 0 0 0 0 0
673 1 0 0 0 0 0
674 1 0 0 0 0 0
675 1 0 0 0 0 0
676 1 0 0 0 0 0
677 1 0 0 0 0 0
678 1 0 0 0 0 0
679 1 0 0 0 0 0
680 1 0 0 0 0 0
681 1 0 0 0 0 0
682 1 0 0 0 0 0
683 1 0 0 0 0 0
684 0 0 0 1 0 222.00320000003558
685 1 0 0 0 0 0
686 1 0 0 0 0 0
687 1 0 0 0 0 0
688 1 0 0 0 0 0
689 1 0 0 0 0 0
690 1 0 0 0 0 0
691 1 0 0 0 0 0
692 1 0 0 0 0 0
693 1 0 0 0 0 0
694 1 0 0 0 0 0
695 1 0 0 0 0 0
696 1 0 0 0 0 0
697 1 0 0 0 0 0
698 1 0 0 0 0 0
699 1 0 0 0 0 0
700 1 0 0 0 0 0
701 1 0 0 0 0 0
702 1 0 0 0 0 0
703 1 0 0 0 0 0
704 1 0 0 0 0 0
705 1 0 0 0 0 0
706 1 0 0 0 0 0
707 1 0 0 0 0 0
708 1 0 0 0 0 0
709 0 1 0 1 0 111.98450000002049
710 1 0 0 0 0 0
711 0 0 0 1 0 190.0134000000544
712 1 0 0 0 0 0
713 1 0 0 0 0 0
714 1 0 0 0 0 0
715 1 0 0 0 0 0
716 1 0 0 0 0 0
717 1 0 0 0 0 0
718 1 0 0 0 0 0
719 1 0 0 0 0 0
720 1 0 0 0 0 0
721 1 0 0 0 0 0
722 1 0 0 0 0 0
723 1 0 0 0 0 0
724 1 0 0 0 0 0
725 1 0 0 0 0 0
726 1 0 0 0 0 0
727 1 0 0 0 0 0
728 1 

1069 0 0 0 1 0 275.98830000008456
1070 1 0 0 0 0 0
1071 0 1 0 1 0 199.96669999998994
1072 1 0 0 0 0 0
1073 0 1 0 1 0 205.98770000017248
1074 0 1 0 1 0 198.00069999997504
1075 1 0 0 0 0 0
1076 1 0 0 0 0 0
1077 1 0 0 0 0 0
1078 0 1 0 1 0 234.00450000003912
1079 1 0 0 0 0 0
1080 1 0 0 0 0 0
1081 0 1 0 1 0 188.00640000007115
1082 0 1 0 1 0 176.00520000001416
1083 0 1 0 1 0 201.97369999997318
1084 1 0 0 0 0 0
1085 1 0 0 0 0 0
1086 0 1 0 1 0 191.9794000000693
1087 1 0 0 0 0 0
1088 0 1 0 1 0 222.003200000152
1089 1 0 0 0 0 0
1090 0 1 0 1 0 199.96659999992698
1091 1 0 0 0 0 0
1092 1 0 0 0 0 0
1093 0 0 0 1 0 181.98519999999553
1094 1 0 0 0 0 0
1095 0 1 0 1 0 210.00190000003204
1096 1 0 0 0 0 0
1097 0 1 0 1 0 246.0057000000961
1098 0 1 0 1 0 183.95140000013635
1099 0 1 0 1 0 204.02169999992475
1100 1 0 0 0 0 0
1101 0 1 0 1 0 183.95139999990351
1102 1 0 0 0 0 0
1103 0 1 0 1 0 192.02040000003763
1104 1 0 0 0 0 0
1105 1 0 0 0 0 0
1106 1 0 0 0 0 0
1107 1 0 0 0 0 0
1108 0 1 0 1 0 193.9865999999456
11

In [77]:
sum(catch)

5